In [63]:
import numpy as np
import pandas as pd
import networkx as nx



import os
import sys
import inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)

sys.path.insert(0, '..')

import Utils as Utils

import scanpy as sc

import mygene

In [65]:
data_path = "scRNA-seq-Gene expression profile at single cell level of a rare seen and young tracheal adenoid cystic carcinoma/";
file_name = "GSE245170_RAW/Tumor/";

adata = sc.read_10x_mtx(data_path+file_name);
adata


AnnData object with n_obs × n_vars = 16325 × 36601
    var: 'gene_ids', 'feature_types'

In [67]:
adata.var["mito"] = adata.var_names.str.startswith("MT-");
sc.pp.filter_cells(adata, min_genes=4000);
sc.pp.filter_genes(adata, min_cells=500);
adata

AnnData object with n_obs × n_vars = 552 × 733
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells'

In [69]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)


In [71]:
sc.pp.highly_variable_genes(adata, n_top_genes=300, inplace=True);
adata


AnnData object with n_obs × n_vars = 552 × 733
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [73]:
adata.var.highly_variable
adata.X
print(adata)
print(adata.X)
display(adata.obs)
display(adata.var)

AnnData object with n_obs × n_vars = 552 × 733
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
<Compressed Sparse Column sparse matrix of dtype 'float32'
	with 390620 stored elements and shape (552, 733)>
  Coords	Values
  (0, 0)	1.2173550128936768
  (1, 0)	0.7845886945724487
  (2, 0)	2.0271241664886475
  (3, 0)	1.6466221809387207
  (5, 0)	0.783635675907135
  (6, 0)	1.2833449840545654
  (7, 0)	1.0149742364883423
  (8, 0)	1.493945837020874
  (9, 0)	2.4101767539978027
  (10, 0)	0.7146300673484802
  (11, 0)	1.42875337600708
  (12, 0)	1.8048014640808105
  (13, 0)	1.9825358390808105
  (14, 0)	1.500443458557129
  (15, 0)	0.9864134192466736
  (16, 0)	1.0863901376724243
  (17, 0)	1.6004642248153687
  (18, 0)	0.7326731085777283
  (19, 0)	1.7489217519760132
  (20, 0)	1.3572174310684204
  (21, 0)	0.82560795545578
  (22, 0)	1.5859206914901733
  (23, 0)	1.2755472660064697
  (24, 0)	1.74

,n_genes
AAATGGAAGAGGCGTT-1,5099
AAATGGAAGTCAGAGC-1,8530
AACAAAGAGGTTCTAC-1,4373
AACAAAGCACTGGATT-1,5238
AACCACAGTCGCATCG-1,5153
...,...
TTTACCAAGTAGGATT-1,4152
TTTACCATCATGCGGC-1,4551
TTTACGTAGTAACCTC-1,8658
TTTAGTCTCGTTGCCT-1,5820


,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
AURKAIP1,ENSG00000175756,Gene Expression,False,532,False,1.586821,0.506955,-0.241670
MRPL20,ENSG00000242485,Gene Expression,False,514,True,1.400645,0.399991,0.120482
RPL22,ENSG00000116251,Gene Expression,False,552,False,2.884979,0.431391,-1.433287
PARK7,ENSG00000116288,Gene Expression,False,552,False,2.186054,0.332958,-1.459384
ENO1,ENSG00000074800,Gene Expression,False,545,True,2.297798,2.674607,1.139357
...,...,...,...,...,...,...,...,...
MT-CO3,ENSG00000198938,Gene Expression,True,552,True,5.538567,3.552707,0.707107
MT-ND3,ENSG00000198840,Gene Expression,True,552,True,3.822957,2.201588,0.239698
MT-ND4,ENSG00000198886,Gene Expression,True,552,False,4.692636,2.842534,-0.020151
MT-ND5,ENSG00000198786,Gene Expression,True,552,False,3.062554,1.632541,-0.209673


In [75]:
file_name = "filtereddata";

adata.write_csvs(data_path+file_name, skip_data=False, sep=',')

In [77]:
obs = pd.read_csv(data_path+file_name+"/obs.csv",index_col=0);
display(obs);
var = pd.read_csv(data_path+file_name+"/var.csv");
display(var);

dataset = pd.read_csv(data_path+file_name+"/X.csv",names=var['gene_ids']);
dataset.index = obs.index;
display(dataset);

highly_variable_genes = var[var['highly_variable']]['gene_ids'].values;
dataset = dataset[highly_variable_genes];
display(dataset);

,n_genes
AAATGGAAGAGGCGTT-1,5099
AAATGGAAGTCAGAGC-1,8530
AACAAAGAGGTTCTAC-1,4373
AACAAAGCACTGGATT-1,5238
AACCACAGTCGCATCG-1,5153
...,...
TTTACCAAGTAGGATT-1,4152
TTTACCATCATGCGGC-1,4551
TTTACGTAGTAACCTC-1,8658
TTTAGTCTCGTTGCCT-1,5820


,Unnamed: 0,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
0,AURKAIP1,ENSG00000175756,Gene Expression,False,532,False,1.586821,0.506955,-0.241670
1,MRPL20,ENSG00000242485,Gene Expression,False,514,True,1.400645,0.399991,0.120482
2,RPL22,ENSG00000116251,Gene Expression,False,552,False,2.884979,0.431391,-1.433287
3,PARK7,ENSG00000116288,Gene Expression,False,552,False,2.186054,0.332958,-1.459384
4,ENO1,ENSG00000074800,Gene Expression,False,545,True,2.297798,2.674607,1.139357
...,...,...,...,...,...,...,...,...,...
728,MT-CO3,ENSG00000198938,Gene Expression,True,552,True,5.538567,3.552707,0.707107
729,MT-ND3,ENSG00000198840,Gene Expression,True,552,True,3.822957,2.201588,0.239698
730,MT-ND4,ENSG00000198886,Gene Expression,True,552,False,4.692636,2.842534,-0.020151
731,MT-ND5,ENSG00000198786,Gene Expression,True,552,False,3.062554,1.632541,-0.209673


,ENSG00000175756,ENSG00000242485,ENSG00000116251,ENSG00000116288,ENSG00000074800,ENSG00000244038,ENSG00000070831,ENSG00000125944,ENSG00000142676,ENSG00000133226,...,ENSG00000198888,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000198886,ENSG00000198786,ENSG00000198727
AAATGGAAGAGGCGTT-1,1.217355,0.000000,2.800846,2.020244,1.938109,1.641309,1.848619,1.217355,4.488579,1.217355,...,4.329608,3.602057,5.339946,4.820878,5.308076,5.395411,3.819732,4.281477,2.352609,4.725415
AAATGGAAGTCAGAGC-1,0.784589,1.303154,3.028192,1.802353,1.583585,1.126595,1.939824,1.698939,4.179624,1.642925,...,3.972711,3.853776,5.779509,5.181438,5.505950,5.563555,3.725963,4.805951,2.765930,5.260275
AACAAAGAGGTTCTAC-1,2.027124,2.027124,2.741026,3.033830,2.898107,1.291128,2.325238,2.325238,3.669566,2.187249,...,3.734612,4.163331,5.627165,5.410829,5.808886,5.704738,3.444392,4.838282,3.309407,5.386984
AACAAAGCACTGGATT-1,1.646622,0.873998,3.022892,1.985698,2.161091,1.129694,2.377258,2.077235,4.270004,1.129694,...,3.364861,3.263334,5.089402,5.163991,4.921120,5.225923,3.457024,4.064837,2.238456,4.890099
AACCACAGTCGCATCG-1,0.000000,1.308364,3.034993,1.308364,1.526089,1.526089,2.208312,1.029646,4.115821,1.308364,...,4.314859,4.115821,5.337580,4.822740,5.420577,5.482346,3.848059,4.674738,2.791205,5.142290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTACCAAGTAGGATT-1,2.419247,0.000000,2.057315,2.057315,1.484434,0.995558,2.684431,2.057315,3.921592,0.000000,...,4.048400,3.694945,5.465780,4.846242,5.581487,5.562017,3.887208,4.604785,2.893828,5.051568
TTTACCATCATGCGGC-1,0.858376,1.313361,2.926983,2.214404,2.474402,1.313361,2.474402,1.862109,3.825544,1.313361,...,4.734653,4.151496,5.778148,5.023124,5.717454,5.761179,3.764429,4.746525,3.385943,5.202853
TTTACGTAGTAACCTC-1,1.726645,1.406385,2.929419,2.192737,2.301261,0.825851,2.038083,1.726645,4.090991,1.406385,...,3.996494,3.757767,5.847446,5.429257,5.523759,5.664070,3.650337,4.801390,3.057975,5.334836
TTTAGTCTCGTTGCCT-1,1.053104,1.249533,2.728513,2.136075,1.053104,0.808419,1.249533,1.249533,4.020908,1.053104,...,2.978858,2.913491,5.129829,4.754478,4.622941,4.682268,3.178912,3.986855,2.768348,4.464305


,ENSG00000242485,ENSG00000074800,ENSG00000117616,ENSG00000142669,ENSG00000090273,ENSG00000158195,ENSG00000184007,ENSG00000127603,ENSG00000117410,ENSG00000132780,...,ENSG00000123562,ENSG00000101856,ENSG00000005893,ENSG00000180879,ENSG00000198888,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000198938,ENSG00000198840
AAATGGAAGAGGCGTT-1,0.000000,1.938109,0.783500,0.783500,1.518935,1.023774,1.023774,1.379471,1.379471,0.783500,...,0.783500,1.023774,1.023774,2.407623,4.329608,3.602057,5.339946,4.820878,5.395411,3.819732
AAATGGAAGTCAGAGC-1,1.303154,1.583585,1.380971,1.380971,1.303154,1.850307,1.453168,2.265757,0.784589,1.218766,...,1.380971,1.583585,1.303154,2.133800,3.972711,3.853776,5.779509,5.181438,5.563555,3.725963
AACAAAGAGGTTCTAC-1,2.027124,2.898107,1.291128,0.000000,1.836378,0.840896,1.600464,2.187249,1.291128,1.600464,...,2.027124,2.446474,0.840896,2.027124,3.734612,4.163331,5.627165,5.410829,5.704738,3.444392
AACAAAGCACTGGATT-1,0.873998,2.161091,0.529591,1.884930,1.333144,0.529591,0.529591,1.129694,1.129694,1.772858,...,0.873998,1.884930,0.000000,2.161091,3.364861,3.263334,5.089402,5.163991,5.225923,3.457024
AACCACAGTCGCATCG-1,1.308364,1.526089,1.856334,1.308364,1.704783,1.526089,1.704783,1.308364,1.308364,1.526089,...,1.704783,1.308364,1.526089,1.308364,4.314859,4.115821,5.337580,4.822740,5.482346,3.848059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTACCAAGTAGGATT-1,0.000000,1.484434,0.995558,2.560604,0.995558,1.484434,1.811349,2.254567,1.484434,1.811349,...,1.811349,2.419247,1.811349,2.057315,4.048400,3.694945,5.465780,4.846242,5.562017,3.887208
TTTACCATCATGCGGC-1,1.313361,2.474402,2.053691,2.474402,0.000000,1.313361,1.624912,0.000000,1.862109,0.858376,...,0.000000,1.862109,2.053691,1.862109,4.734653,4.151496,5.778148,5.023124,5.761179,3.764429
TTTACGTAGTAACCTC-1,1.406385,2.301261,1.271906,1.197215,1.271906,1.341404,1.771306,2.133714,1.679896,1.341404,...,1.855056,1.197215,1.406385,2.070987,3.996494,3.757767,5.847446,5.429257,5.664070,3.650337
TTTAGTCTCGTTGCCT-1,1.249533,1.053104,1.977102,0.000000,1.249533,1.053104,1.249533,1.977102,0.483770,1.053104,...,0.483770,1.977102,1.249533,1.887010,2.978858,2.913491,5.129829,4.754478,4.682268,3.178912


In [79]:
gene_list = dataset.columns.tolist();

mg = mygene.MyGeneInfo();

geneSyms = mg.querymany(gene_list, scopes='ensembl.gene', fields='symbol', species='human');

gene_info_dict = {};
for ele in geneSyms:
    # print(ele);
    query = ele['query'];
    if 'symbol' in ele:
        symbol = ele['symbol'];
        gene_info_dict[query] = symbol;
    
dataset.rename(columns=gene_info_dict, inplace=True);
dataset

dataset.to_csv(data_path+file_name+"/processed_data.csv",index=None);

1 input query terms found no hit:	['ENSG00000112096']
